In [ ]:
import geopandas as gpd

# Read the GeoPackage file
file_path = "inputs/Asentamientos humanos 2/Polígonos/CELDAS_fixed.gpkg"
gdf = gpd.read_file(file_path)

# Display the first few rows of the GeoDataFrame
gdf.head()

In [ ]:
gdf.info()

In [ ]:
gdf.crs

In [ ]:
gdf_epsg_4326 = gdf.to_crs("EPSG:4326")

In [ ]:
gdf_epsg_4326.to_parquet("outputs/celdas_02_06_2025.parquet", index=False)

In [ ]:
import urbanpy as up

In [ ]:
lima = up.download.nominatim_osm("Lima, Peru", 2, email="claudio@autodash.org")

In [ ]:
huancabamba = up.download.nominatim_osm(
    "Huancabamba, Peru", 0, email="claudio@autodash.org"
)

In [ ]:
huancabamba.crs

In [ ]:
huancabamba.plot()

In [ ]:
celdas = gdf_epsg_4326

In [ ]:
celdas.columns

In [ ]:
celdas["pop_2020"].describe()

In [ ]:
celdas[celdas["pop_2020"] == 0].shape

In [ ]:
celdas[celdas["pop_2020"] == 0]

In [ ]:
celdas["smod"].value_counts()

In [ ]:
celdas["smod"].value_counts(normalize=True) * 100

In [ ]:
celdas[celdas["smod"] != "rural"].shape

In [ ]:
celdas[(celdas["pop_2020"] > 1) & (celdas["smod"] != "rural")]["pop_2020"].describe()

In [ ]:
celdas_huancabamba = celdas[celdas.intersects(huancabamba.geometry[0])]

In [ ]:
celdas_huancabamba.shape

In [ ]:
import matplotlib.pyplot as plt
import contextily as ctx

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

huancabamba.plot(ax=ax, facecolor="none", edgecolor="black")
celdas_huancabamba.plot(
    "pop_2020", cmap="viridis", legend=True, ax=ax, alpha=0.5, edgecolor="black"
)

In [ ]:
# Schools
schools, metadata = up.download.overpass(
    type_of_data="node",
    mask=huancabamba,
    query={
        "amenity": "school",
    },
)

In [ ]:
schools.shape

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))

huancabamba.plot(ax=ax, facecolor="none", edgecolor="black")
celdas_huancabamba.plot(
    "pop_2020", cmap="viridis", legend=True, ax=ax, alpha=0.5, edgecolor="black"
)
schools.plot(ax=ax, color="red", markersize=10)

# Add a basemap
# ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=huancabamba.crs)

In [ ]:
# start server
up.routing.start_osrm_server("peru", "south-america", "foot")